# Random Forest

In [315]:
import pandas as pd
import numpy as np
import joblib
from taxipred.utils.constants import DATA_PATH
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

df = pd.read_csv(DATA_PATH / "taxi_cleaned.csv")

df.sample(5)

,Trip_Distance_km,Passenger_Count,Base_Fare,Per_Km_Rate,Per_Minute_Rate,Trip_Duration_Minutes,Trip_Price,Time_of_Day_Evening,Time_of_Day_Morning,Time_of_Day_Night,Day_of_Week_Weekend,Traffic_Conditions_Low,Traffic_Conditions_Medium,Weather_Rain,Weather_Snow
232,25.23,3.0,3.94,1.35,0.13,100.22,51.0291,False,True,False,False,True,False,True,False
237,31.29,1.0,2.74,1.14,0.34,24.70,46.8086,False,False,False,True,False,True,False,True
23,27.79,2.0,2.77,0.72,0.43,49.63,44.1197,True,False,False,False,True,False,False,False
116,33.22,1.0,2.31,1.66,0.32,41.02,70.5816,False,False,True,True,True,False,False,False
140,28.93,2.0,3.31,2.00,0.46,29.14,74.5744,False,True,False,False,False,True,False,False


### 0. Divide into X and y

In [301]:
df_train = df.dropna(subset=["Trip_Price"])

X, y = df_train.drop("Trip_Price", axis="columns"), df_train["Trip_Price"]

X.head()

,Trip_Distance_km,Passenger_Count,Base_Fare,Per_Km_Rate,Per_Minute_Rate,Trip_Duration_Minutes,Time_of_Day_Evening,Time_of_Day_Morning,Time_of_Day_Night,Day_of_Week_Weekend,Traffic_Conditions_Low,Traffic_Conditions_Medium,Weather_Rain,Weather_Snow
0,19.35,3.0,3.56,0.80,0.32,53.82,False,True,False,False,True,False,False,False
1,36.87,1.0,2.70,1.21,0.15,37.27,True,False,False,True,False,False,False,False
2,8.64,2.0,2.55,1.71,0.48,89.33,False,False,False,True,False,True,False,False
4,41.79,3.0,4.60,1.77,0.11,86.95,False,False,True,True,False,False,False,False
5,9.91,2.0,2.32,1.26,0.34,41.72,True,False,False,False,False,False,False,False


In [302]:
y.info()

<class 'pandas.core.series.Series'>
Index: 548 entries, 0 to 573
Series name: Trip_Price
Non-Null Count  Dtype  
--------------  -----  
548 non-null    float64
dtypes: float64(1)
memory usage: 8.6 KB


### 1. train|test split

In [303]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42
    )


### 3. Training

In [304]:
rf = RandomForestRegressor(random_state=42)
rf.fit(X_train, y_train)

,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",100
,"criterion criterion: {""squared_error"", ""absolute_error"", ""friedman_mse"", ""poisson""}, default=""squared_error""The function to measure the quality of a split. Supported criteriaare ""squared_error"" for the mean squared error, which is equal tovariance reduction as feature selection criterion and minimizes the L2loss using the mean of each terminal node, ""friedman_mse"", which usesmean squared error with Friedman's improvement score for potentialsplits, ""absolute_error"" for the mean absolute error, which minimizesthe L1 loss using the median of each terminal node, and ""poisson"" whichuses reduction in Poisson deviance to find splits.Training using ""absolute_error"" is significantly slowerthan when using ""squared_error""... versionadded:: 0.18 Mean Absolute Error (MAE) criterion... versionadded:: 1.0 Poisson criterion.",'squared_error'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",None
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: {""sqrt"", ""log2"", None}, int or float, default=1.0The number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None or 1.0, then `max_features=n_features`... note:: The default of 1.0 is equivalent to bagged trees and more randomness can be achieved by setting smaller values, e.g. 0.3... versionchanged:: 1.1 The default of `max_features` changed from `""auto""` to 1.0.Note: the search for a split does not stop until at least onevalid partition of the node samples is found, even if it requires toeffectively inspect more than ``max_features`` features.",1.0
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow trees with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsample

### 4. Predicting

In [305]:
y_pred = rf.predict(X_test)
y_pred

array([ 46.281754,  34.946871, 102.318708,  25.730642,  44.091335,
        50.882911,  49.402187,  34.588209,  30.941044,  39.827266,
       105.779765,  46.123221,  31.332178,  29.491834,  42.765976,
        82.522437,  40.4826  ,  69.205304,  20.011384,  54.671497,
        61.295588,  55.182684,  25.584825,  59.502899,  30.983214,
        58.354099,  54.385416,  36.964867,  41.383545,  27.845756,
        28.412723,  56.01576 ,  21.980646,  95.265134,  81.658553,
        62.799993,  59.084416,  36.435164,  51.80871 ,  12.517858,
        92.982957,  70.809646,  13.302272,  59.392894,  67.916423,
        79.705404,  33.500541,  70.102228,  40.370164,  25.414   ,
        53.218338,  17.676556,  70.841614,  75.09063 ,  89.19813 ,
        29.187772,  11.050866,  48.273579,  40.479712,  18.644526,
        56.601863,  17.172163,  45.061504,  71.288503,  75.108073,
        56.644336,  65.758946,  85.007165,  56.147784,  42.426359,
        16.973582,  36.468534,  68.224406,  54.425589,  50.319

### 5. Evaluation

In [306]:
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

mae, mse, rmse

(3.7945711712707153, 24.98193167729187, np.float64(4.998192841146874))

### RandomForestRegression gave us RMSE = 5.00

Feature importance

In [307]:
rf.feature_importances_

array([0.49833533, 0.00377259, 0.01178319, 0.25853154, 0.07330673,
       0.14453681, 0.00155757, 0.00107998, 0.00084602, 0.00143996,
       0.00119405, 0.00128034, 0.00158   , 0.00075589])

In [308]:
feature_importance = pd.DataFrame([X.columns, rf.feature_importances_]).T
feature_importance.columns = ["feature", "importance"]
feature_importance = feature_importance.sort_values(by="importance", ascending=False)
feature_importance

,feature,importance
0,Trip_Distance_km,0.498335
3,Per_Km_Rate,0.258532
5,Trip_Duration_Minutes,0.144537
4,Per_Minute_Rate,0.073307
2,Base_Fare,0.011783
1,Passenger_Count,0.003773
12,Weather_Rain,0.00158
6,Time_of_Day_Evening,0.001558
9,Day_of_Week_Weekend,0.00144
11,Traffic_Conditions_Medium,0.00128


### KNN

### 2. Scaling dataset

In [309]:
scaler = StandardScaler()
scaler.fit(X_train)

scaled_X_train = scaler.transform(X_train)
scaled_X_test = scaler.transform(X_test)

scaled_X_train.min(), scaled_X_train.max(), scaled_X_test.min(), scaled_X_test.max()

(np.float64(-1.7360722288089256),
 np.float64(3.3516165253998453),
 np.float64(-1.716131388651957),
 np.float64(3.3516165253998453))

### 3. Training

In [310]:
model = KNeighborsRegressor(n_neighbors=5)
model.fit(scaled_X_train, y_train)

,"n_neighbors n_neighbors: int, default=5Number of neighbors to use by default for :meth:`kneighbors` queries.",5
,"weights weights: {'uniform', 'distance'}, callable or None, default='uniform'Weight function used in prediction. Possible values:- 'uniform' : uniform weights. All points in each neighborhood are weighted equally.- 'distance' : weight points by the inverse of their distance. in this case, closer neighbors of a query point will have a greater influence than neighbors which are further away.- [callable] : a user-defined function which accepts an array of distances, and returns an array of the same shape containing the weights.Uniform weights are used by default.See the following example for a demonstration of the impact ofdifferent weighting schemes on predictions::ref:`sphx_glr_auto_examples_neighbors_plot_regression.py`.",'uniform'
,"algorithm algorithm: {'auto', 'ball_tree', 'kd_tree', 'brute'}, default='auto'Algorithm used to compute the nearest neighbors:- 'ball_tree' will use :class:`BallTree`- 'kd_tree' will use :class:`KDTree`- 'brute' will use a brute-force search.- 'auto' will attempt to decide the most appropriate algorithm based on the values passed to :meth:`fit` method.Note: fitting on sparse input will override the setting ofthis parameter, using brute force.",'auto'
,"leaf_size leaf_size: int, default=30Leaf size passed to BallTree or KDTree. This can affect thespeed of the construction and query, as well as the memoryrequired to store the tree. The optimal value depends on thenature of the problem.",30
,"p p: float, default=2Power parameter for the Minkowski metric. When p = 1, this isequivalent to using manhattan_distance (l1), and euclidean_distance(l2) for p = 2. For arbitrary p, minkowski_distance (l_p) is used.",2
,"metric metric: str, DistanceMetric object or callable, default='minkowski'Metric to use for distance computation. Default is ""minkowski"", whichresults in the standard Euclidean distance when p = 2. See thedocumentation of `scipy.spatial.distance`_ andthe metrics listed in:class:`~sklearn.metrics.pairwise.distance_metrics` for valid metricvalues.If metric is ""precomputed"", X is assumed to be a distance matrix andmust be square during fit. X may be a :term:`sparse graph`, in whichcase only ""nonzero"" elements may be considered neighbors.If metric is a callable function, it takes two arrays representing 1Dvectors as inputs and must return one value indicating the distancebetween those vectors. This works for Scipy's metrics, but is lessefficient than passing the metric name as a string.If metric is a DistanceMetric object, it will be passed directly tothe underlying computation routines.",'minkowski'
,"metric_params metric_params: dict, default=NoneAdditional keyword arguments for the metric function.",None
,"n_jobs n_jobs: int, default=NoneThe number of parallel jobs to run for neighbors search.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details.Doesn't affect :meth:`fit` method.",None


### 4. Predicting

In [311]:
y_pred_knn = model.predict(scaled_X_test)
y_pred_knn

array([34.68142, 37.35814, 86.93886, 31.39116, 44.0573 , 57.13154,
       52.73846, 43.43216, 45.66176, 56.47006, 85.08226, 72.60282,
       34.01706, 33.1681 , 33.20804, 49.86396, 53.40362, 74.05942,
       28.15162, 70.77734, 44.32356, 50.62904, 58.2023 , 61.1888 ,
       54.92822, 59.29056, 54.28084, 53.23018, 50.90856, 47.92202,
       33.28822, 49.93612, 29.28534, 68.02032, 63.53386, 52.98224,
       51.44574, 41.2538 , 37.83932, 23.72902, 70.2663 , 59.3227 ,
       21.56672, 50.86836, 51.7192 , 58.1848 , 43.26136, 43.0582 ,
       30.59618, 39.09782, 34.66232, 32.14676, 69.81758, 82.54776,
       70.93302, 37.62444, 19.85032, 54.28314, 66.94708, 34.98878,
       60.9143 , 33.60238, 46.95304, 71.27992, 75.7842 , 33.9704 ,
       70.03674, 66.82884, 47.41128, 50.92334, 20.9986 , 33.41082,
       80.05066, 49.94062, 50.70982, 69.57586, 62.4182 , 69.46356,
       54.21812, 44.44612, 44.25394, 57.44442, 40.32798, 65.29182,
       45.30976, 38.39766, 50.7717 , 47.4711 , 69.26724, 39.14

### 5. Evaluation

In [312]:
mae_knn = mean_absolute_error(y_test, y_pred_knn)
mse_knn = mean_squared_error(y_test, y_pred_knn)
rmse_knn = np.sqrt(mse_knn)

mae_knn, mse_knn, rmse_knn

(11.0532964640884, 191.125459230221, np.float64(13.824813171620837))

### KNN gave us RMSE = 13.82

# Conclution

- RandomForestRegression gave us RMSE = 5.00
- KNN gave us RMSE = 13.82


# Training model

In [316]:
rf.fit(X, y)

,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",100
,"criterion criterion: {""squared_error"", ""absolute_error"", ""friedman_mse"", ""poisson""}, default=""squared_error""The function to measure the quality of a split. Supported criteriaare ""squared_error"" for the mean squared error, which is equal tovariance reduction as feature selection criterion and minimizes the L2loss using the mean of each terminal node, ""friedman_mse"", which usesmean squared error with Friedman's improvement score for potentialsplits, ""absolute_error"" for the mean absolute error, which minimizesthe L1 loss using the median of each terminal node, and ""poisson"" whichuses reduction in Poisson deviance to find splits.Training using ""absolute_error"" is significantly slowerthan when using ""squared_error""... versionadded:: 0.18 Mean Absolute Error (MAE) criterion... versionadded:: 1.0 Poisson criterion.",'squared_error'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",None
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: {""sqrt"", ""log2"", None}, int or float, default=1.0The number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None or 1.0, then `max_features=n_features`... note:: The default of 1.0 is equivalent to bagged trees and more randomness can be achieved by setting smaller values, e.g. 0.3... versionchanged:: 1.1 The default of `max_features` changed from `""auto""` to 1.0.Note: the search for a split does not stop until at least onevalid partition of the node samples is found, even if it requires toeffectively inspect more than ``max_features`` features.",1.0
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow trees with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsample

In [ ]:
joblib.dump(rf, "taxi_price_prediction.joblib", compress=("xz", 3), protocol=5)

['taxi_prices_prediction.joblib']